In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from zipfile import ZipFile
import os
import pandas as pd

In [ ]:
with ZipFile('dogs-vs-cats-redux-kernels-edition.zip') as zip_ref:
    zip_ref.extractall('ref_zip')
with ZipFile('ref_zip/train.zip') as zip_ref:
    zip_ref.extractall()

        filename label
0   cat.9920.jpg   cat
1  cat.11403.jpg   cat
2   cat.2180.jpg   cat
3   cat.1109.jpg   cat
4    dog.696.jpg   dog


In [10]:
filenames = os.listdir('train')
labels = ['dog' if 'dog' in name else 'cat' for name in filenames]
df = pd.DataFrame({'filename' : filenames, 'label' : labels})
print(df.head())

        filename label
0   cat.9920.jpg   cat
1  cat.11403.jpg   cat
2   cat.2180.jpg   cat
3   cat.1109.jpg   cat
4    dog.696.jpg   dog


In [16]:
#Image transformations
from torchvision import transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomAffine(degrees=15, shear = 20, scale=(0.9, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
    ])
val_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
    ])

In [17]:
from torch.utils.data import Dataset
from PIL import Image
import os

In [18]:
# Make custom dataset // 이미지텐서, 라벨텐서로 변환하는 과정
class Dataframedataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.dataframe = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = 1 if self.dataframe.iloc[idx, 1] == 'dog' else 0
        label = torch.tensor(label, dtype=torch.float32)

        if self.transform:
            image = self.transform(image)

        return image, label

In [19]:
# split Dataset
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42, 
    stratify=df['label']
    )

In [26]:
train_dataset = Dataframedataset(
    dataframe=train_df,
    img_dir='train',
    transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = Dataframedataset(
    dataframe=val_df,
    img_dir='train',
    transform=val_transform
)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.features = nn.Sequential(

            # Block 1
            nn.Conv2d(3, 32, kernel_size=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),

            # Block 2
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),

            # Block 3
            nn.Conv2d(64, 128, kernel_size=3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),

            # Block 4
            nn.Conv2d(128, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),

            # Block 5
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.2),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),  # GlobalAveragePooling2D
            nn.Flatten(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [22]:
device = torch.device('cuda')
model = cnn().to(device)

In [23]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}")
    print(f"Train Loss: {epoch_loss:.4f}")
    

0.6928642392158508
0.6931735277175903
0.6935638189315796
0.6938836574554443
0.6937358975410461
0.6927077770233154
0.693352460861206
0.6934780478477478
0.6932791471481323
0.6923754215240479
0.6929730176925659
0.6931666135787964
0.6930387020111084
0.6932159662246704
0.6926853656768799
0.6932103037834167
0.6933635473251343
0.693763256072998
0.6926742792129517
0.6930367350578308
0.692642092704773
0.693423867225647
0.6947299242019653
0.6931946873664856
0.6932802200317383
0.6931039094924927
0.6935046315193176
0.6935506463050842
0.6927250623703003
0.6924716234207153
0.6933488845825195
0.6932476758956909
0.693446159362793
0.6927430033683777
0.6932916641235352
0.69349205493927
0.6930440664291382
0.693681538105011
0.6927134990692139
0.6922562122344971
0.6926798224449158
0.6938877105712891
0.6931319236755371
0.6932991743087769
0.6929515600204468
0.6930413246154785
0.6925520896911621
0.693518340587616
0.6939603090286255
0.6919467449188232
0.6930292248725891
0.6921777725219727
0.6913622617721558
0.

KeyboardInterrupt: 